In [5]:
import pandas as pd
import numpy as np
import riskfolio 
import matplotlib.pyplot as plt
import pypfopt as ppo
import sqlite3
import os

In [40]:
ibov_close = pd.read_csv('./data/ativos_ibov/ivob_close.csv')

In [41]:
threshold = len(ibov_close) 
ibov_close.dropna(thresh=threshold, axis=1, inplace=True)
ibov_close

,Date,BBAS3.SA,BBDC3.SA,BRAP4.SA,BRFS3.SA,BRKM5.SA,CMIG4.SA,CPFE3.SA,CPLE6.SA,CSNA3.SA,...,GOAU4.SA,ITSA4.SA,PETR3.SA,PETR4.SA,SBSP3.SA,TRPL4.SA,USIM5.SA,VALE3.SA,VIVT3.SA,WEGE3.SA
0,2000-01-03 00:00:00,0.493556,1.018616,0.767667,749.570984,1384.787354,2527.665527,0.110521,0.561978,1.707281,...,141.599884,0.396037,1.333409,1.364051,4.741245,2120.581055,1.459322,1.436180,7.079099,0.119891
1,2000-01-04 00:00:00,0.470082,0.944535,0.712833,744.739929,1350.556885,2401.087891,0.130615,0.540104,1.731846,...,136.717087,0.358225,1.258575,1.288593,4.460146,1851.301025,1.406117,1.401985,6.887372,0.115684
2,2000-01-05 00:00:00,0.474898,0.976636,0.767667,744.739929,1356.780884,2414.137695,0.130615,0.545152,1.824843,...,134.764023,0.388077,1.275583,1.275591,4.336030,2053.260254,1.539888,1.401985,6.964061,0.115684
3,2000-01-06 00:00:00,0.481518,0.991453,0.767667,725.393677,1425.242065,2446.106934,0.130615,0.535056,1.907312,...,136.228867,0.394047,1.265378,1.271180,4.314290,2022.967041,1.679740,1.401985,7.117442,0.105167
4,2000-01-07 00:00:00,0.470684,1.008738,0.767667,725.393677,1462.585083,2453.285156,0.140663,0.530009,1.895030,...,136.717087,0.392057,1.275583,1.276984,4.374958,1952.280518,1.746625,1.453277,7.152839,0.105167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6215,2024-09-30 00:00:00,27.180000,12.893271,20.219999,23.650000,19.930000,11.400000,33.930000,10.020653,12.890000,...,10.920000,11.070000,39.270000,36.009998,90.400002,24.299999,6.240000,63.509998,55.820000,54.389999
6216,2024-10-01 00:00:00,27.080000,12.962749,20.230000,24.280001,20.040001,11.420000,33.860001,10.060000,13.050000,...,10.870000,10.890000,40.320000,36.970001,90.029999,24.260000,6.240000,63.820000,55.570000,55.880001
6217,2024-10-02 00:00:00,27.150000,13.380000,20.280001,23.860001,20.280001,11.400000,33.950001,10.150000,13.300000,...,11.100000,10.930000,40.790001,37.480000,90.110001,24.430000,6.310000,64.169998,56.090000,55.349998
6218,2024-10-03 00:00:00,26.900000,13.250000,19.830000,23.740000,19.820000,11.240000,33.810001,9.950000,12.920000,...,10.750000,10.600000,41.340000,37.939999,88.720001,24.080000,6.110000,62.939999,54.689999,54.599998


In [47]:
import riskfolio as rp

# Convert 'Date' column to datetime
ibov_close['Date'] = pd.to_datetime(ibov_close['Date'])
ibov_close.set_index('Date', inplace=True)

# Define the portfolio object
port = rp.Portfolio(returns=ibov_close)

# Estimate the expected returns and covariance matrix
port.assets_stats(method_mu='hist', method_cov='hist')

# Define the optimization model
model = 'Classic'  # Could be 'Classic', 'BL', 'FM'
rm = 'MV'  # Risk measure used, here 'MV' stands for Minimum Volatility
obj = 'MinRisk'  # Objective function
hist = True  # Use historical scenarios for risk measures that depend on scenarios

# Rebalance every two weeks
rebalance_dates = pd.date_range(start=ibov_close.index[0], end=ibov_close.index[-1], freq='2W')
weights = []

for date in rebalance_dates:
    # Slice the data up to the current rebalance date
    data = ibov_close.loc[:date]
    
    # Update the portfolio object with the sliced data
    port = rp.Portfolio(returns=data)
    port.assets_stats(method_mu='hist', method_cov='hist')
    
    # Optimize the portfolio
    w = port.optimization(model=model, rm=rm, obj=obj, hist=hist)
    weights.append(w)

# Combine weights into a DataFrame
weights_df = pd.DataFrame(weights, index=rebalance_dates, columns=ibov_close.columns)
print(weights_df)

C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\stats\moment_helpers.py:252: RuntimeWarning: invalid value encountered in divide
  corr = cov / np.outer(std_, std_)


You must convert self.cov to a positive definite matrix
You must convert self.cov to a positive definite matrix
You must convert self.cov to a positive definite matrix


ValueError: Must pass 2-d input. shape=(646, 23, 1)